In [1]:
import os
import pandas as pd
import numpy as np
from tqdm import *

In [2]:
allImages = os.listdir('../BigDataSource/Teddy2024/附件3/ImageData')
allImages

['Image14001016-4576.jpg',
 'Image14105001-8071.jpg',
 'Image14105002-0481.jpg',
 'Image14105001-0439.jpg',
 'Image14001016-0704.jpg',
 'Image14105001-8065.jpg',
 'Image14001014-2197.jpg',
 'Image14105003-4803.jpg',
 'Image14105004-5952.jpg',
 'Image14105001-4111.jpg',
 'Image14105001-6706.jpg',
 'Image14105003-8787.jpg',
 'Image14001015-3289.jpg',
 'Image14105002-9499.jpg',
 'Image14105001-1928.jpg',
 'Image14105004-5013.jpg',
 'Image14001014-3276.jpg',
 'Image14105002-6741.jpg',
 'Image14105002-0456.jpg',
 'Image14105004-1513.jpg',
 'Image14001014-1307.jpg',
 'Image14001014-7762.jpg',
 'Image14105003-0330.jpg',
 'Image14001014-5175.jpg',
 'Image14105002-6033.jpg',
 'Image14105002-8778.jpg',
 'Image14001016-5867.jpg',
 'Image14105004-3476.jpg',
 'Image14105002-4142.jpg',
 'Image14105002-3605.jpg',
 'Image14105004-5991.jpg',
 'Image14001015-9001.jpg',
 'Image14001016-3594.jpg',
 'Image14105003-3605.jpg',
 'Image14105002-3177.jpg',
 'Image14105001-6538.jpg',
 'Image14001015-4914.jpg',
 

In [3]:
allImagesList = pd.read_csv('../BigDataSource/Teddy2024/附件3/image_test.csv')
allImagesList.shape

(5000, 1)

In [4]:
import chardet
from modules.models.SBERT import Text2Vec

file_name = '../BigDataSource/Teddy2024/附件3/word_data.csv'
# 首先，读取文件的一部分字节来检测编码
with open(file_name, 'rb') as f:
    raw_data = f.read(5000)
    result = chardet.detect(raw_data)
    encoding = result['encoding']
allTextList = pd.read_csv(file_name, encoding=encoding)
allTextList = np.array(allTextList)

def process_word(inputs):
    word, text_transfer = inputs
    features = text_transfer.text2vec(word)
    return features


text2vec = Text2Vec()
inputList = [(allTextList[i, 1], text2vec) for i in range(len(allTextList))]
from concurrent.futures import ThreadPoolExecutor, as_completed
# Execute
all_text_features = []
with ThreadPoolExecutor(max_workers=16) as executor:
    futures = [executor.submit(process_word, inputs) for inputs in inputList]
    for future in tqdm(as_completed(futures), total=len(inputList)):
        all_text_features.append(future.result())
print('Done!')

/Users/zengyuxiang/opt/anaconda3/lib/python3.8/site-packages/scipy/__init__.py:143: UserWarning: A NumPy version >=1.19.5 and <1.27.0 is required for this version of SciPy (detected version 1.19.2)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
100%|██████████| 50000/50000 [08:18<00:00, 100.26it/s]

Done!


In [5]:
import pickle
with open('../BigDataSource/Teddy2024/附件3/word_features.pkl', 'wb') as f:
    pickle.dump(all_text_features, f)

In [6]:
# allImagesFeatures = []
# for i in trange(len(allImages)):
#     imageTranfer = Image2tensor()
#     file_name = '../BigDataSource/Teddy2024/附件3/ImageData/' + allImages[i]
#     image = Image.open(file_name)
#     features = imageTranfer.image2tensor(image)
#     allImagesFeatures.append(features)
# allImagesFeatures

In [7]:
from modules.models.ResNet50 import Image2tensor
from PIL import Image

# 这里是执行多线程代码，线性运算过慢，线性预算代码在上面，用时8mins
from concurrent.futures import ThreadPoolExecutor, as_completed

# function
def process_image(inputs):
    imageTranfer, image_name = inputs
    file_name = '../BigDataSource/Teddy2024/附件3/ImageData/' + image_name
    image = Image.open(file_name)
    features = imageTranfer.image2tensor(image)
    return features

# input
allImages = os.listdir('../BigDataSource/Teddy2024/附件3/ImageData')
imageTranfer = Image2tensor()
inputList = [(imageTranfer, imageNames) for imageNames in allImages]

# Execute
allImagesFeatures = []
allImages = os.listdir('../BigDataSource/Teddy2024/附件3/ImageData')
with ThreadPoolExecutor(max_workers=16) as executor:
    futures = [executor.submit(process_image, inputs) for inputs in inputList]
    for future in tqdm(as_completed(futures), total=len(allImages)):
        allImagesFeatures.append(future.result())
print('Done!')

/Users/zengyuxiang/opt/anaconda3/lib/python3.8/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'dlopen(/Users/zengyuxiang/opt/anaconda3/lib/python3.8/site-packages/torchvision/image.so, 0x0006): Symbol not found: __ZN3c1017RegisterOperatorsD1Ev
  Referenced from: <70619B86-59ED-3849-B040-4C861402850E> /Users/zengyuxiang/opt/anaconda3/lib/python3.8/site-packages/torchvision/image.so
  Expected in:     <9E86230E-21B4-3734-AB52-949AE1F40CB9> /Users/zengyuxiang/opt/anaconda3/lib/python3.8/site-packages/torch/lib/libtorch_cpu.dylib'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
100%|██████████| 5000/5000 [01:43<00:00, 48.29it/s]

Done!


In [8]:
# 将所有变量转成numpy形式
allImagesFeatures_now = []
for i in range(len(allImagesFeatures)):
    imageFeatures = np.array(allImagesFeatures[i]).reshape(-1, )
    allImagesFeatures_now.append(imageFeatures)
allImagesFeatures = np.array(allImagesFeatures_now)
allImagesFeatures.shape

(5000, 1000)

In [9]:
# 将所有变量转成numpy形式
allTextFeatures_now = []
for i in range(len(allTextList)):
    textFeatures = np.array(allTextList[i, 1]).reshape(-1, )
    allTextFeatures_now.append(textFeatures)
allTextFeatures = np.array(allTextFeatures_now)
allTextFeatures.shape

(50000, 1)

In [10]:
allTextFeatures.shape, allImagesFeatures.shape

((50000, 1), (5000, 1000))

In [11]:
from sklearn.decomposition import PCA
import numpy as np

# 假设 `visual_features` 是一个形状为 (n_samples, 1000) 的数组，

# 初始化PCA对象，设置目标维度为384
pca = PCA(n_components=384)

# 对视觉特征进行降维
scaled_allImagesFeatures = pca.fit_transform(allImagesFeatures)

# 查看降维后的形状，确认是否为 (n_samples, 384)
print("降维后的视觉特征形状:", scaled_allImagesFeatures.shape)


降维后的视觉特征形状: (5000, 384)


In [12]:
xb = scaled_allImagesFeatures
xq = allTextFeatures

In [13]:
from modules.models.Retrieve import LSH, L2Index
from time import time 

d = 384  # 向量维度
k = 5
np.random.seed(1234)  # 保证可复现的结果

t1 = time()
lsh = LSH(k=k, d=d, nbits=4)
topk_distence, topk_indices = lsh.search_topk_embeds(xb, xq)
t2 = time()
print(f'LSH: {t2 - t1 : .2f}s')


ValueError: could not convert string to float: '洛阳楼盘 老城区楼盘 道北楼盘 保利<人名>'

In [ ]:
topk_indices

In [ ]:
xb = scaled_allImagesFeatures
xq = allTextFeatures

In [ ]:

t1 = time()
l2index = L2Index(k=k, d=d)
topk_distence, topk_indices = l2index.search_topk_embeds(xb, xq)
t2 = time()
print(f'L2: {t2 - t1 : .2f}s')

In [ ]:
topk_indices